In [1]:
import pandas as pd
import numpy as np
rrh = pd.read_csv('race-result-horse.csv')
rrh.shape

(30189, 19)

In [2]:
#  drop the rows where the “finish_position” is not a number (e.g. WV-A, WV). There may be accidents.
rrh = rrh[rrh.finishing_position.isin(['1','2','3','4','5','6','7','8','9','10','11','12','13','14'])].reset_index()
rrh.head()

,index,finishing_position,horse_number,horse_name,horse_id,jockey,trainer,actual_weight,declared_horse_weight,draw,length_behind_winner,running_position_1,running_position_2,running_position_3,running_position_4,finish_time,win_odds,running_position_5,running_position_6,race_id
0,0,1,1.0,DOUBLE DRAGON,K019,B Prebble,D Cruz,133,1032,1,-,1.0,2.0,2.0,1.0,1.22.33,3.8,NaN,NaN,2014-001
1,1,2,2.0,PLAIN BLUE BANNER,S070,D Whyte,D E Ferraris,133,1075,13,2,8.0,9.0,9.0,2.0,1.22.65,8,NaN,NaN,2014-001
2,2,3,10.0,GOLDWEAVER,P072,Y T Cheng,Y S Tsui,121,1065,3,2,2.0,1.0,1.0,3.0,1.22.66,5.7,NaN,NaN,2014-001
3,3,4,3.0,SUPREME PROFIT,P230,J Moreira,C S Shum,132,1222,2,2,6.0,4.0,5.0,4.0,1.22.66,6.1,NaN,NaN,2014-001
4,4,5,7.0,THE ONLY KID,H173,Z Purton,K W Lui,125,1136,9,4-1/4,9.0,10.0,10.0,5.0,1.23.02,6.1,NaN,NaN,2014-001


In [3]:
horse_id = rrh.horse_id.unique()
no_horse = len(horse_id)
horse_index = range(no_horse)

jockey = rrh.jockey.unique()
no_jockey = len(jockey)
jockey_index = range(no_jockey)

trainer = rrh.trainer.unique()
no_trainer = len(trainer)
trainer_index = range(no_trainer)

print(' Number of horses: ' , no_horse , '\n', 'Number of Jokeys: ' , no_jockey ,'\n', 'Number of Trainers: ' , no_trainer)

 Number of horses:  2155 
 Number of Jokeys:  105 
 Number of Trainers:  93


In [4]:
import warnings
warnings.filterwarnings("ignore")

In [5]:
# Takes a while to run

# Add a column named recent_6_runs to the dataframe, which records the recent ranks of the horse in each entry. 
# The ranks are separated by “/”, and a record is like 1/2/6/3/4/7.

# Add a column named recent_ave_rank for each entry to the dataframe, 
# which records the average rank of the recent 6 runs of a horse

# rrh['recent_6_runs'] = '0'
# rrh['recent_ave_rank'] = '7'

# for i in range(len(rrh['finishing_position'])):
    
#     temp = rrh[ : (i + 1)][rrh.horse_id == rrh.horse_id[i]][['finishing_position']]
#     temp = temp['finishing_position'].values.tolist()[::-1]
  
#     rrh['recent_6_runs'][i] = '/'.join(temp[:6])
    
#     if len(temp) != 0:
#         temp_int = map(int,temp)    # convert string to integer
#         temp_ave = np.mean(list(temp_int))
#         rrh['recent_ave_rank'][i] = temp_ave

In [6]:
# Add column of Distance

# The distance could be 1000, 1200, 1400, 1600, 1800, 2000, 2400, etc. 
# Some horses are good at short-distance races, while some are good at long-distance races.

rrh['race_distance'] = '0'
rrr = pd.read_csv('race-result-race.csv')
for i in range(len(rrr['race_id'])):
    distance = rrr['race_distance'][i]
    rrh['race_distance'][rrh.race_id == rrr.race_id[i]] = distance

In [7]:
# Add HorseWin, HorseRankTop3, HorseRankTop50Percent for use in Part3

rrh['HorseWin'] = (rrh.finishing_position == '1') + 0
rrh['HorseRankTop3'] = (rrh.finishing_position == '1') + (rrh.finishing_position == '2') + (rrh.finishing_position == '3') + 0

index=rrh.index[rrh['finishing_position'] == '1'].tolist()

rrh['HorseRankTop50Percent'] = '0'
for i in range(len(index)-1):
    rrh['HorseRankTop50Percent'][index[i]:(index[i] + int(round(0.5 * (index[i + 1] - index[i]))))] = '1'
rrh['HorseRankTop50Percent'][index[len(index) - 1]:(index[len(index) - 1] + 6)] = '1'

In [8]:
# Divide data into training set and test set.
# Training set contains all the races with race_id no more than “2016-327” (first 80%), 
# Test set contains the rest of races (last 20%)

temp_index = max(rrh.index[rrh['race_id'] == '2016-327'].tolist())
training = rrh[:(temp_index + 1)]
testing = rrh[(temp_index + 1):]

In [30]:
# # Calculate jockey_ave_rank, trainer_ave_rank for training data
# # jockey_ave_rank: records the average rank of the jockey in the training data
# # trainer_ave_rank:records the average rank of the trainer in the training data
# # # Add jockey record

jockAvg = np.zeros(len(training))
jockey = np.unique(training["jockey"])

for i in range(len(jockey)):
    temp = training[training.jockey == jockey[i]]
    tempI = temp.index
    if (i%10)==0:
        print("{}/{}".format(i, len(jockey)))
    
    for j in range(len(temp)):
        temp_2 = temp.iloc[j,:]
        date = temp_2["race_id"]
        
        beforeRaces = temp.loc[temp["race_id"]<date]
        
        if beforeRaces.shape[0] != 0:
            vals = [int(i) for i in beforeRaces["finishing_position"].values]
            jockAvg[tempI[j]] = np.mean(vals)
            if jockAvg[tempI[j]] > 1000000000:
                print(jockAvg[tempI[j]])

training['jockey_ave_rank'] = jockAvg

trainAvg = np.zeros(len(training))
trainer = np.unique(training["trainer"])

for i in range(len(trainer)):
    temp = training[training.trainer == trainer[i]]
    tempI = temp.index
    if (i%10)==0:
        print("{}/{}".format(i, len(trainer)))
    
    for j in range(len(temp)):
        temp_2 = temp.iloc[j,:]
        date = temp_2["race_id"]
        
        beforeRaces = temp.loc[temp["race_id"]<date]
        
        if beforeRaces.shape[0] != 0:
            vals = [int(i) for i in beforeRaces["finishing_position"].values] 
            trainAvg[tempI[j]] = np.mean(vals)

training['trainer_ave_rank'] = trainAvg

recent = np.zeros(len(training))

horses = np.unique(training["horse_id"])
for i in range(len(horses)):
    temp = training[training.horse_id == horses[i]]
    tempI = temp.index
    if (i%10)==0:
        print("{}/{}".format(i, len(horses)))
    
    for j in range(len(temp)):
        temp_2 = temp.iloc[j,:]
        date = temp_2["race_id"]
        
        beforeRaces = temp.loc[temp["race_id"]<date]
        
        if beforeRaces.shape[0] != 0:
            vals = [int(i) for i in beforeRaces["finishing_position"].values]
            if beforeRaces.shape[0] <= 6:
                recent[tempI[j]] = np.mean(vals)
            else:
                recent[tempI[j]] = np.mean(vals[-6:])

training['recent_ave_rank'] = recent

training.to_csv('training.csv')

0/99
10/99
20/99
30/99
40/99
50/99
60/99
70/99
80/99
90/99
0/87
10/87
20/87
30/87
40/87
50/87
60/87
70/87
80/87
0/1907
10/1907
20/1907
30/1907
40/1907
50/1907
60/1907
70/1907
80/1907
90/1907
100/1907
110/1907
120/1907
130/1907
140/1907
150/1907
160/1907
170/1907
180/1907
190/1907
200/1907
210/1907
220/1907
230/1907
240/1907
250/1907
260/1907
270/1907
280/1907
290/1907
300/1907
310/1907
320/1907
330/1907
340/1907
350/1907
360/1907
370/1907
380/1907
390/1907
400/1907
410/1907
420/1907
430/1907
440/1907
450/1907
460/1907
470/1907
480/1907
490/1907
500/1907
510/1907
520/1907
530/1907
540/1907
550/1907
560/1907
570/1907
580/1907
590/1907
600/1907
610/1907
620/1907
630/1907
640/1907
650/1907
660/1907
670/1907
680/1907
690/1907
700/1907
710/1907
720/1907
730/1907
740/1907
750/1907
760/1907
770/1907
780/1907
790/1907
800/1907
810/1907
820/1907
830/1907
840/1907
850/1907
860/1907
870/1907
880/1907
890/1907
900/1907
910/1907
920/1907
930/1907
940/1907
950/1907
960/1907
970/1907
980/1907
990/1907

In [32]:
####----Refer jockey_ave_rank and trainer_ave_rank of testing data to training data---####

# Calculate jockey_ave_rank, trainer_ave_rank for training data
# jockey_ave_rank: records the average rank of the jockey in the training data
# trainer_ave_rank:records the average rank of the trainer in the training data
# # Add jockey record
testing = testing.reset_index(drop=True)
jockAvg = np.zeros(len(testing))
jockey = np.unique(testing["jockey"])

for i in range(len(jockey)):
    temp = testing[testing.jockey == jockey[i]]
    tempI = temp.index

    if (i%10)==0:
        print("{}/{}".format(i, len(jockey)))
    
    for j in range(len(temp)):
        temp_2 = temp.iloc[j,:]
        date = temp_2["race_id"]
        
        beforeRaces = temp.loc[temp["race_id"]<date]
        
        if beforeRaces.shape[0] != 0:
            vals = [int(i) for i in beforeRaces["finishing_position"].values]
            jockAvg[tempI[j]] = np.mean(vals)

testing['jockey_ave_rank'] = jockAvg

trainAvg = np.zeros(len(testing))
trainer = np.unique(testing["trainer"])

for i in range(len(trainer)):
    temp = testing[testing.trainer == trainer[i]]
    tempI = temp.index
    if (i%10)==0:
        print("{}/{}".format(i, len(trainer)))
    
    for j in range(len(temp)):
        temp_2 = temp.iloc[j,:]
        date = temp_2["race_id"]
        
        beforeRaces = temp.loc[temp["race_id"]<date]
        
        if beforeRaces.shape[0] != 0:
            vals = [int(i) for i in beforeRaces["finishing_position"].values]
            trainAvg[tempI[j]] = np.mean(vals)

testing['trainer_ave_rank'] = trainAvg

recent = np.zeros(len(testing))

horses = np.unique(testing["horse_id"])
for i in range(len(horses)):
    temp = testing[testing.horse_id == horses[i]]
    tempI = temp.index
    if (i%10)==0:
        print("{}/{}".format(i, len(horses)))
    
    for j in range(len(temp)):
        temp_2 = temp.iloc[j,:]
        date = temp_2["race_id"]
        
        beforeRaces = temp.loc[temp["race_id"]<date]
        
        if beforeRaces.shape[0] != 0:
            vals = [int(i) for i in beforeRaces["finishing_position"].values]
            if beforeRaces.shape[0] <= 6:
                recent[tempI[j]] = np.mean(vals)
            else:
                recent[tempI[j]] = np.mean(vals[-6:])

testing['recent_ave_rank'] = recent

0/43
10/43
20/43
30/43
40/43
0/34
10/34
20/34
30/34
0/1148
10/1148
20/1148
30/1148
40/1148
50/1148
60/1148
70/1148
80/1148
90/1148
100/1148
110/1148
120/1148
130/1148
140/1148
150/1148
160/1148
170/1148
180/1148
190/1148
200/1148
210/1148
220/1148
230/1148
240/1148
250/1148
260/1148
270/1148
280/1148
290/1148
300/1148
310/1148
320/1148
330/1148
340/1148
350/1148
360/1148
370/1148
380/1148
390/1148
400/1148
410/1148
420/1148
430/1148
440/1148
450/1148
460/1148
470/1148
480/1148
490/1148
500/1148
510/1148
520/1148
530/1148
540/1148
550/1148
560/1148
570/1148
580/1148
590/1148
600/1148
610/1148
620/1148
630/1148
640/1148
650/1148
660/1148
670/1148
680/1148
690/1148
700/1148
710/1148
720/1148
730/1148
740/1148
750/1148
760/1148
770/1148
780/1148
790/1148
800/1148
810/1148
820/1148
830/1148
840/1148
850/1148
860/1148
870/1148
880/1148
890/1148
900/1148
910/1148
920/1148
930/1148
940/1148
950/1148
960/1148
970/1148
980/1148
990/1148
1000/1148
1010/1148
1020/1148
1030/1148
1040/1148
1050/1148

In [33]:
testing.to_csv('testing.csv')

In [ ]:
testing.reindex(range(len(testing))).index

In [11]:
testing.head()

,index,finishing_position,horse_number,horse_name,horse_id,jockey,trainer,actual_weight,declared_horse_weight,draw,...,running_position_4,finish_time,win_odds,running_position_5,running_position_6,race_id,race_distance,HorseWin,HorseRankTop3,HorseRankTop50Percent
23500,24155,1,5.0,POWERMAX,A009,N Callan,R Gibson,126,1124,9,...,NaN,1.10.69,6,NaN,NaN,2016-328,1200,1,1,1
23501,24156,2,2.0,BUDDY BUNDY,T157,K K Chiong,D Cruz,127,1193,8,...,NaN,1.10.70,16,NaN,NaN,2016-328,1200,0,1,1
23502,24157,3,3.0,ROYALE ELEGANCE,S098,O Murphy,W Y So,131,1090,1,...,NaN,1.10.76,4.5,NaN,NaN,2016-328,1200,0,1,1
23503,24158,4,11.0,CONFUCIUS SPIRIT,S329,K Teetan,D E Ferraris,120,1073,2,...,NaN,1.10.89,5.5,NaN,NaN,2016-328,1200,0,0,1
23504,24159,5,10.0,STARLIGHT,V273,Z Purton,C H Yip,122,1056,12,...,NaN,1.10.91,14,NaN,NaN,2016-328,1200,0,0,1


In [12]:
t = testing

In [14]:
testing.reset_index()

,level_0,index,finishing_position,horse_number,horse_name,horse_id,jockey,trainer,actual_weight,declared_horse_weight,...,running_position_4,finish_time,win_odds,running_position_5,running_position_6,race_id,race_distance,HorseWin,HorseRankTop3,HorseRankTop50Percent
0,23500,24155,1,5.0,POWERMAX,A009,N Callan,R Gibson,126,1124,...,NaN,1.10.69,6,NaN,NaN,2016-328,1200,1,1,1
1,23501,24156,2,2.0,BUDDY BUNDY,T157,K K Chiong,D Cruz,127,1193,...,NaN,1.10.70,16,NaN,NaN,2016-328,1200,0,1,1
2,23502,24157,3,3.0,ROYALE ELEGANCE,S098,O Murphy,W Y So,131,1090,...,NaN,1.10.76,4.5,NaN,NaN,2016-328,1200,0,1,1
3,23503,24158,4,11.0,CONFUCIUS SPIRIT,S329,K Teetan,D E Ferraris,120,1073,...,NaN,1.10.89,5.5,NaN,NaN,2016-328,1200,0,0,1
4,23504,24159,5,10.0,STARLIGHT,V273,Z Purton,C H Yip,122,1056,...,NaN,1.10.91,14,NaN,NaN,2016-328,1200,0,0,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5859,29359,30182,9,7.0,INTREPIC,A139,M F Poon,D J Hall,116,993,...,9.0,1.24.05,4,NaN,NaN,2016-805,1400,0,0,0
5860,29360,30183,10,12.0,HIGH AND MIGHTY,S362,D Whyte,W Y So,119,1132,...,10.0,1.24.18,7.2,NaN,NaN,2016-805,1400,0,0,0
5861,29361,30184,11,3.0,DOUBLE VALENTINE,A163,B Prebble,A S Cruz,127,1085,...,11.0,1.24.58,41,NaN,NaN,2016-805,1400,0,0,0
5862,29362,30185,12,8.0,THE JOY OF GIVING,A249,W M Lai,C W Chang,122,1026,...,12.0,1.25.26,99,NaN,NaN,2016-805,1400,0,0,0


level_0                  False
index                    False
finishing_position       False
horse_number             False
horse_name               False
horse_id                 False
jockey                   False
trainer                  False
actual_weight            False
declared_horse_weight    False
draw                     False
length_behind_winner     False
running_position_1       False
running_position_2       False
running_position_3       False
running_position_4        True
finish_time              False
win_odds                 False
running_position_5        True
running_position_6        True
race_id                  False
race_distance            False
HorseWin                 False
HorseRankTop3            False
HorseRankTop50Percent    False
jockey_ave_rank          False
trainer_ave_rank         False
recent_ave_rank          False
dtype: bool